In [59]:
import numpy as np
import pandas as pd


In [62]:
# 1.初始化参数
def initialize_parameters(n_x, n_z, n_y):
    np.random.seed(2)

    # 权重和偏置矩阵
    w1 = np.random.randn(n_z, n_x) * 0.01
    b1 = np.zeros(shape=(n_z, 1))
    w2 = np.random.randn(n_y, n_z) * 0.01
    b2 = np.zeros(shape=(n_y, 1))

    # 通过字典存储参数
    parameters = {'w1': w1, 'b1': b1, 'w2': w2, 'b2': b2}

    return parameters

In [63]:
#定义Sigmoid函数
def sigmoid(x):
    return 1.0 / (1.0+np.exp(-x))

In [64]:
# 2.前向传播
def forward_propagation(X, parameters):
    w1 = parameters['w1']
    b1 = parameters['b1']
    w2 = parameters['w2']
    b2 = parameters['b2']

    # 通过前向传播来计算a2
    z1 = np.dot(w1, X) + b1     # 这个地方需注意矩阵加法：虽然(w1*X)和b1的维度不同，但可以相加
    a1 = sigmoid(z1)            # 使用sigmoid作为第一层的激活函数
    z2 = np.dot(w2, a1) + b2
    a2 = sigmoid(z2)  # 使用sigmoid作为第二层的激活函数

    # 通过字典存储参数
    cache = {'z1': z1, 'a1': a1, 'z2': z2, 'a2': a2}

    return a2, cache


In [65]:

# 3.反向传播
def backward_propagation(parameters, cache, X, Y):
    m = Y.shape[1]

    w1 = parameters['w1']
    w2 = parameters['w2']

    a1 = cache['a1']
    a2 = cache['a2']

    # 反向传播，计算dw1、db1、dw2、db2
    dz2 = a2 - Y
    dw2 = (1 / m) * np.dot(dz2, a1.T)
    db2 = (1 / m) * np.sum(dz2, axis=1, keepdims=True)
    dz1 = np.multiply(np.dot(w2.T, dz2), 1 - np.power(a1, 2))
    dw1 = (1 / m) * np.dot(dz1, X.T)
    db1 = (1 / m) * np.sum(dz1, axis=1, keepdims=True)

    grads = {'dw1': dw1, 'db1': db1, 'dw2': dw2, 'db2': db2}

    return grads


In [66]:
# 4.更新参数
def update_parameters(parameters, grads, learning_rate=0.8):
    w1 = parameters['w1']
    b1 = parameters['b1']
    w2 = parameters['w2']
    b2 = parameters['b2']

    dw1 = grads['dw1']
    db1 = grads['db1']
    dw2 = grads['dw2']
    db2 = grads['db2']

    # 更新参数
    w1 = w1 - dw1 * learning_rate
    b1 = b1 - db1 * learning_rate
    w2 = w2 - dw2 * learning_rate
    b2 = b2 - db2 * learning_rate

    parameters = {'w1': w1, 'b1': b1, 'w2': w2, 'b2': b2}

    return parameters



In [67]:

# 建立神经网络
def nn_model(X, Y, n_h, n_input, n_output, num_iterations=10000):
    np.random.seed(3)

    n_x = n_input           # 输入层节点数
    n_y = n_output          # 输出层节点数

    # 1.初始化参数
    parameters = initialize_parameters(n_x, n_h, n_y)

    # 梯度下降循环
    for i in range(0, num_iterations):
        # 2.前向传播
        a2, cache = forward_propagation(X, parameters)

        # 4.反向传播
        grads = backward_propagation(parameters, cache, X, Y)
        # 5.更新参数
        parameters = update_parameters(parameters, grads)
   
       
    return parameters


In [68]:
# 对模型进行测试
def predict(parameters, x_test, y_test):
    w1 = parameters['w1']
    b1 = parameters['b1']
    w2 = parameters['w2']
    b2 = parameters['b2']

    z1 = np.dot(w1, x_test) + b1
    a1 = np.tanh(z1)
    z2 = np.dot(w2, a1) + b2
    a2 = 1 / (1 + np.exp(-z2))

    # 结果的维度
    n_rows = y_test.shape[0]
    n_cols = y_test.shape[1]

    # 预测值结果存储
    output = np.empty(shape=(n_rows, n_cols), dtype=int)

    for i in range(n_rows):
        for j in range(n_cols):
            if a2[i][j] > 0.5:
                output[i][j] = 1
            else:
                output[i][j] = 0

    print('预测结果：')
    print(output)
    print('真实结果：')
    print(y_test)

    count = 0
    for k in range(0, n_cols):
        if output[0][k] == y_test[0][k] and output[1][k] == y_test[1][k] and output[2][k] == y_test[2][k]:
            count = count + 1
        else:
            print(k)

    acc = count / int(y_test.shape[1]) * 100
    print('准确率：%.2f%%' % acc)


    acc = count / int(y_test.shape[1]) * 100
    print('准确率：%.2f%%' % acc)
	
   

In [69]:
if __name__ == "__main__":
    # 读取数据
    data_set = pd.read_csv(r'E:\Jupyter_notebooks\data\iris_training1.csv',header=None)
    X = data_set.iloc[:, 0:4].values.T          # 前四列是特征，T表示转置
    Y = data_set.iloc[:, 4:].values.T           # 后三列是标签

    Y = Y.astype('uint8')
 
    #开始训练
    parameters = nn_model(X, Y, n_h=6, n_input=4, n_output=3, num_iterations=10000)
  
    # 对模型进行测试
    data_test = pd.read_csv(r'E:\Jupyter_notebooks\data\iris_test1.csv',header=None)
    x_test = data_test.iloc[:, 0:4].values.T
    y_test = data_test.iloc[:, 4:].values.T
    y_test = y_test.astype('uint8')

    result = predict(parameters, x_test, y_test)

C:\Users\沈君君\AppData\Local\Temp\ipykernel_30728\418872950.py:3: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0+np.exp(-x))


预测结果：
[[0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0]
 [1 0 0 1 1 1 0 1 1 0 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 1 0 1]
 [0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0]]
真实结果：
[[0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0]
 [1 0 0 1 1 1 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 1 0 1]
 [0 1 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0]]
7
准确率：96.67%
准确率：96.67%
